In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns=999
train=pd.read_csv('../input/home-data-for-ml-course/train.csv')
train


In [ ]:
test=pd.read_csv('../input/home-data-for-ml-course/test.csv')
test


In [ ]:
all_data=pd.concat([train,test])
all_data2=all_data.drop(columns=['Id','SalePrice'])

선형모델 -> 데이터에 민감하기 때문에 전처리에 조금 더 신경쓰기
1. 문자를 숫자로 바꾸는 과정 
-특정 칼럼 레이블링 
-선형 모델 시에는 숫자를 거리로 처리해버려서 제대로 안 나오게 됨/ 더미 변수로 문자형 만들어주는 원 핫 인코딩으로 바꾸기 
eg. 사과 카테고리 컬럼을 세 개로 만들어서 더미 변수로 (1,0,0) (0,1,0) (0,0,1) -> 알아서 문자형 .카테고리형 칼럼에 접근해서 다가옴 
2. 중요한 칼럼에 주로 가중치를 높게 두겠지? ㅇㅇ 그러면 가중치가 가장 큰 값을 이용해서 판단 ->regularization을 통해서 파악
3. 모든 칼럼을 동등하게 판단이 ㄴㄴ하니까 -> 데이터마다 칼럼의 범위가 다르니까 -> 범위를 스케일링화 해준다. 정규분포는 아니지만 범위를 조정함
4. 내부적으로 점수 좋아지는지 어케 체크? 평가용 데이터 셋을 차리기-> train셋과 평가 셋 나누기(과대적합(평가셋에 대한 불확신,대표성이 없음 편향된 데이터일 수 있다),학습 시 데이터가 충분하지 않아서 손해이다) -> train_test_split이 문제가 되니까 교차 검증으로 해서 안정적으로 학습시키자 


In [ ]:
all_data3=pd.get_dummies(all_data2)
all_data3

In [ ]:
all_data3=all_data3.fillna(-1)

In [ ]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
all_data4=st.fit_transform(all_data3)
all_data4

In [ ]:
all_data4=pd.DataFrame(all_data4,columns=all_data3.columns)
all_data4

In [ ]:
train_2=all_data4[:len(train)]
train_2

In [ ]:
test_2=all_data4[len(train):]
test_2

Ridge는 L2 규제를 사용한다. L1 규제는 필요없는 칼럼은 아얘 사용 안함 (Lasso)

In [ ]:
from sklearn.linear_model import Ridge
Rg=Ridge(alpha=10) #규제를 하지 않으면 점수 개선 ㄴㄴ임, 특정 칼럼에 더 가중치를 줄거다 
Rg.fit(train_2,train['SalePrice'])
result=Rg.predict(test_2)

In [ ]:
sub=pd.read_csv('../input/home-data-for-ml-course/sample_submission.csv')
sub['SalePrice']=result
sub

In [ ]:
sub.to_csv('house8.csv',index=0)